In [1]:
import pandas as pd

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [3]:
finance_news = pd.read_excel('finance_data/SmoothNLP36kr新闻数据集10k.xlsx')
finance_news = finance_news.drop(columns = ['Unnamed: 0'])

In [4]:
finance_news.columns

Index(['content', 'insert_ts', 'pub_ts', 'title', 'url'], dtype='object')

In [5]:
finance_news['content'].head()

0    如果你有一个产品，这个产品很有可能颠覆整个行业，但是这个产品可能需要你酝酿很久才能赚到钱，你...
1    三星Galaxy S III这款手机已经吊足了很多人的胃口。最近国外网站\nTweakers...
2    今天晚上，甲骨文向Google索赔61亿美元的世纪侵权大案\n将在旧金山联邦法院开审。预计8...
3    Pinterest是一款很棒的产品，不仅一举成为\n全美第三大社交网站，围绕\n图片战略衍生...
4    当苹果、Amazon以及它们的竞争对手们在全球就电子书定价问题打得不可开交之时，一家快速崛起...
Name: content, dtype: object

In [47]:
finance_news['content'].iloc[2]

'今天晚上，甲骨文向Google索赔61亿美元的世纪侵权大案\n将在旧金山联邦法院开审。预计8周的时间后，将有一方潸然泪下。\n审理过程中，Google CEO Larry Page、甲骨文CEO Larry Ellison、前Sun公司CEO Jonathan Schwartz和Google Android总裁Andy Rubin都将出庭作证。\n该案审理的结果可能会产生非常深远的意义。我们一起来理清一下具体的情况：\n专利问题\n甲骨文最先于10年夏起诉Google称其拥有的Android系统侵犯了7项Java专利，索赔61亿美元。随后在专利检查中，有5项被剔除出去了，最后留下2项有争议的专利。\nGoogle此前曾希望甲骨文放弃诉讼，作为回报其将抽出一部分Android所获利润再加上280万美元的一次性赔偿来补偿甲骨文（总计约1400万美元）。当然前提是甲骨文要在庭审中证明Android确实侵犯了2项Java专利，不过甲骨文拒绝了。\n版权问题\n甲骨文还将控诉Google在开发Android的过程中复制了超过37项Java API（application programming interfaces）以及11行源代码。这一控诉对于程序员来说具有深远影响，其判决结果将直接意味着今后开发者能否复制其他产品的API 。\nGoogle则认为API不应该适用版权保护法，因为其更类似开发者用来开发软件的工具。更为简单的说就是Google认为一段具体的程序可以受到版权保护，而用来写程序的语言则不应该受到版权保护。\n以上便是该诉讼的一些背景，相信庭审开始后会有更多的消息出来。\n更新1（2012.5.24）：陪审团判决Android并未侵犯甲骨文专利，第2阶段审判结束。接下来将进行第3阶段审判，双方将主要就版权问题再度激战。\n图片来源：\nclash of titans'

In [7]:
import jieba
jieba.enable_paddle()

Paddle enabled successfully......


In [8]:
tokens = jieba.cut("所以，整个过程有点像勘探。首先是有人发起，然后有千百人买了镐和铁锹跟进。有人会满载而归，但看走眼的人也会空手回来。"
                   ,cut_all=False)

In [49]:
from zhon.hanzi import punctuation as punctuation_zh
from string import punctuation
import collections
import re

In [50]:
def read_stop_words():
    stop_words = set()
    with open('words_utils/stopwords','r') as file:
        for word in file:
            word = word.strip()
            stop_words.add(word)
    return stop_words

In [51]:
stop_words = read_stop_words()

In [52]:
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [65]:
class JiebaTokenizer():
    def __init__(self, stop_words, punctuations, vocab = None):
        self.stop_words = stop_words
        self.punctuation = set()
        for punctuation in punctuations:
            self.punctuation.update(set(punctuation))
        self.vocab = self.init_vocab(vocab)
            
    def init_vocab(self, vocab):
        init_vocab = {'<pad>':0,
                 '<unk>':1,
                 'CLS':2,
                 'SEP':3,
                 'MASK':4}
        init_vocab = collections.OrderedDict(init_vocab)
        if vocab is not None:
            for word in vocab:
                init_vocab[word] = len(init_vocab)
        return init_vocab
    
    def tokenize(self, text):
        tokens = []
        for token in jieba.cut(text,cut_all=False):
            if token in self.stop_words:
                continue
            if token in self.punctuation:
                continue
            if token == ' ' or token == '\n' or token[0].isnumeric():
                continue
            if len(token) == 1 and token[0].isalpha():
                continue
            word = token.lower()
            tokens.append(word)
            self.vocab[word] = len(self.vocab)
        return tokens

In [66]:
tokenizer = JiebaTokenizer(stop_words, [punctuation, punctuation_zh])

In [71]:
def write_data_txt(finance_news):
    with open('finance_data/data.txt','w') as file:
        for index, row in finance_news.iterrows():
            content = row['content']
            try:
                file.write(content)
                file.write('\n\n')
            except Exception as e:
                print(content, e)
                continue

In [72]:
write_data_txt(finance_news)

nan write() argument must be str, not float
nan write() argument must be str, not float


In [73]:
import json
from albertlib import albert_model, tokenization
from albertlib.create_pretraining_data import create_training_instances, write_instance_to_example_files
from albertlib.albert import AlbertConfig
from albertlib.input_pipeline import create_pretrain_dataset
import random
from absl import logging, flags
import tensorflow as tf

In [74]:
train_meta_data = {
    "max_seq_length": 50,
    "max_predictions_per_seq": 10
}

In [75]:
logging.set_verbosity(logging.INFO)
FLAGS = flags.FLAGS
FLAGS.meta_data_file_path = 'processed_data/train_meta_data'
FLAGS.input_file = 'finance_data/data.txt'
FLAGS.max_seq_length = train_meta_data['max_seq_length']
FLAGS.max_predictions_per_seq = train_meta_data['max_predictions_per_seq']
FLAGS.output_file = 'processed_data/train.tf_record'
FLAGS.dupe_factor = 40
FLAGS.masked_lm_prob = 0.15
FLAGS.short_seq_prob = 0.1
FLAGS.mark_as_parsed()

tokenizer = JiebaTokenizer(stop_words, [punctuation, punctuation_zh])

input_files = []
for input_pattern in FLAGS.input_file.split(","):
    input_files.extend(tf.io.gfile.glob(input_pattern))

logging.info("*** Reading from input files ***")
for input_file in input_files:
    logging.info("  %s", input_file)

rng = random.Random(FLAGS.random_seed)
instances = create_training_instances(
  input_files, tokenizer, FLAGS.max_seq_length, FLAGS.dupe_factor,
  FLAGS.short_seq_prob, FLAGS.masked_lm_prob, FLAGS.max_predictions_per_seq,
  rng)

logging.info("number of instances: %i", len(instances))

output_files = FLAGS.output_file.split(",")
logging.info("*** Writing to output files ***")
for output_file in output_files:
    logging.info("  %s", output_file)

write_instance_to_example_files(instances, tokenizer, FLAGS.max_seq_length,
                              FLAGS.max_predictions_per_seq, output_files)

INFO:absl:*** Reading from input files ***
INFO:absl:  finance_data/data.txt
  0%|          | 0/1 [00:00<?, ?it/s]

KeyboardInterrupt: 